# workerノードの追加

---

CoursewareHub環境を構成しているworkerノードに新たなノードを追加します。

## パラメータの指定

workerノードを追加するのに必要となるパラメータを入力します。

### UnitGroup名

操作対象となるVCPのUnitGroup名を指定します。

VCノードを作成時に指定したUnitGroup名を確認するために `group_vars` ファイル名の一覧を表示します。

In [ ]:
!ls -1 --hide all group_vars/

UnitGroup名を指定してください。

In [ ]:
# (例)
# ugroup_name = 'CoursewareHub'

ugroup_name = 

#### チェック

対象となるVCノードがAnsibleによって操作できることを確認します。

Ansibleの設定ファイルの場所を環境変数に設定しておきます。

In [ ]:
from pathlib import Path
import os

cfg_ansible = Path("ansible.cfg")
if cfg_ansible.exists():
    os.environ["ANSIBLE_CONFIG"] = str(cfg_ansible.resolve())

managerノードにアクセスできることを確認します。

In [ ]:
target_hub = f"{ugroup_name}_manager"

!ansible {target_hub} -m ping

UnitGroup名に対応する group_varsファイルが存在していることを確認します。

In [ ]:
!test -f group_vars/{ugroup_name}

UnitGroupの変数をgroup_varsファイルから読み込みます。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(ugroup_name)

### VCCアクセストークンの入力

VCノードを追加するためにVC Controller(VCC)のアクセストークンが必要となります。
次のセルを実行すると表示される入力枠にVCCのアクセストークンを入力してください。

> アクセストークン入力後に Enter キーを押すことで入力が完了します。

In [ ]:
from getpass import getpass

vcc_access_token = getpass()

入力されたアクセストークンが正しいことを、実際にVCCにアクセスして確認します。

In [ ]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK

vcp = VcpSDK(vcc_access_token)

上のセルの実行結果がエラーとなり以下のようなメッセージが表示されている場合は、入力されたアクセストークンに誤りがあります。

```
config vc failed: http_status(403)
2023/XX/XX XX:XX:XX UTC: VCPAuthException: xxxxxxx:token lookup is failed: permission denied
```

エラーになった場合はこの節のセルを全て `unfreeze` してから、もう一度アクセストークンの入力を行ってください。

### 追加するノードの指定

workerノードから追加するノードを指定します。

追加するノードの指定は二通りの方法で行うことができます。

* 追加するworkerノード数を指定する
* 追加するノードのIPアドレスを指定する

ノードの追加を行う前に、現在のworkerノードの状態を確認します。

In [ ]:
ugroup = vcp.get_ugroup(ugroup_name)
unit_worker = ugroup.get_unit("worker")
unit_worker.df_nodes()

#### 追加するworkerノード数を指定する

> 追加するノードをIPアドレスで指定する場合はこの節をスキップしてください。

追加するノード数を指定してください。

In [ ]:
# (例)
# add_nodes = 2

add_nodes = 

#### 追加するノードのIPアドレスを指定する

> 追加するノード数を指定した場合はこの節をスキップしてください。ノード数とIPアドレスの両方を指定した場合はIPアドレスの指定が優先されます。

追加するノードのIPアドレスのリストを指定してください。

In [ ]:
# (例)
# add_ipaddresses = [
#     '172.30.2.101',
#     '172.30.2.102',
#     '172.30.2.103',
# ]

add_ipaddresses = [
    
]

#### 追加するノードのMACアドレスを指定する

> 追加するノード数を指定した場合、またはMACアドレスの指定が不要な場合はこの節をスキップしてください。ノード数とIPアドレスの両方を指定した場合はIPアドレスの指定が優先されます。またIPアドレスとMACアドレスの両方を指定する場合は、同じ数のアドレスを指定してください。

追加するノードのMACアドレスのリストを指定してください。

In [ ]:
# (例)
# add_mac_addresses = [
#     '4a:d1:4d:ab:cf:14',
#     '4a:d1:4d:ab:cf:15',
#     '4a:d1:4d:ab:cf:16',
# ]

add_mac_addresses = [

]

#### チェック

指定されたパラメータをチェックします。

In [ ]:
worker_addrs = unit_worker.find_ip_addresses()
if "add_ipaddresses" in vars() and len(add_ipaddresses) > 0:
    if len(set(add_ipaddresses) & set(worker_addrs)) > 0:
        raise RuntimeError("既存のIPアドレスと重複した値が指定されています")
    if "add_mac_addresses" in vars() and len(add_mac_addresses) > 0 and len(add_mac_addresses) != len(add_ipaddresses):
        raise RuntimeError("指定されたIPアドレス数とMACアドレス数が異なります")
elif "add_nodes" in vars():
    if add_nodes <= 0:
        raise RuntimeError("追加するノード数には正の値を指定してください")

## ノードの追加

### NFSサーバ

NFSサーバが属しているansibleのグループ名を指定します。

In [ ]:
if 'nfs_target' in gvars:
    nfs_group = gvars['nfs_target']
elif 'nfs_ipaddress' in gvars:
    nfs_group = f'{ugroup_name}_nfs'
else:
    nfs_group = f'{ugroup_name}_manager'

指定されたグループのノードを操作できることを確認します。

In [ ]:
!ansible {nfs_group} -m ping

In [ ]:
!ansible {nfs_group} -b -a 'whoami'

### /etc/exportsの設定を一時変更する

追加するノードからNFSサーバにアクセスできるようにするためにサブネット全体を許可するように`/etc/exports`の設定を変更します。ノードを追加後にworkerノードのIPアドレスが確定した後に、workerノードからのみアクセス可能となるように`/etc/exports`を設定しなおします。

一時的にNFSサーバへのアクセスを許可するサブネットの値を確認します。

In [ ]:
print(vcp.get_vpn_catalog(gvars["vc_provider"]).get("private_network_ipmask"))

`/etc/exports`の記述を変更します。

In [ ]:
from tempfile import TemporaryDirectory
from pathlib import Path

exports_opts = 'rw,fsid=0,no_root_squash,no_subtree_check,sync,crossmnt'
subnet = vcp.get_vpn_catalog(gvars["vc_provider"]).get('private_network_ipmask')
with TemporaryDirectory() as workdir:
        exports = Path(workdir) / f'{ugroup_name}.exports'
        with exports.open(mode='w') as f:
               f.write(f'/exported/{ugroup_name} {subnet}({exports_opts})\n')
        !cat {exports}
        !ansible {nfs_group} -b -D -m copy -a \
                'src={exports} dest=/etc/exports.d/ backup=yes'

設定ファイルを読み込ませてエクスポート設定を更新します。

In [ ]:
!ansible {nfs_group} -b -a 'exportfs -r -v'

### VCノードの追加

VCノードを追加します。

In [ ]:
params = {}
if "add_ipaddresses" in vars() and len(add_ipaddresses) > 0:
    params["ip_addresses"] = add_ipaddresses
if "add_mac_addresses" in vars() and len(add_mac_addresses) > 0:
    params["mac_addresses"] = add_mac_addresses
if "add_nodes" in vars():
    params["num_add_nodes"] = add_nodes
unit_worker.add_nodes(**params)

追加後のworkerノードの状態を確認します。

In [ ]:
unit_worker.df_nodes()

### Ansibleの設定を更新する

Ansibleのインベントリにノードの情報を追加します。

In [ ]:
from pathlib import Path
import yaml

inventory_path = Path("inventory.yml")
with inventory_path.open() as f:
    inventory = yaml.safe_load(f)
inventory["all"]["children"][ugroup_name]["children"][
    f"{ugroup_name}_{unit_worker.name}"
]["hosts"] = dict([(x, dict(servicenet_ip=x)) for x in unit_worker.find_ip_addresses()])
bak_inventory_path = Path(inventory_path.parent, inventory_path.name + ".bak")
inventory_path.rename(bak_inventory_path)
with inventory_path.open(mode="w") as f:
    yaml.safe_dump(inventory, f)

変更差分を確認します。

In [ ]:
! ! diff -u {bak_inventory_path} {inventory_path}

追加したVCノードにSSHでログインできるようにするために `~/.ssh/known_hosts` の更新を行います。

In [ ]:
from time import sleep


def check_update_known_hosts(ipaddr):
    # VCノード起動直後だと sshd サービスが開始されておらずに known_hosts が更新されない場合がある
    # ssh-keyscan が値を取得できるまで何度かリトライする
    for x in range(10):
        out = ! echo $(ssh-keyscan {ipaddr} 2> /dev/null | wc -l)
        update_lines = int(out[0])
        if update_lines > 0:
            break
        sleep(1)
    else:
        raise RuntimeError("ERROR: timeout!")


!mkdir -p -m 0700 ~/.ssh
!touch ~/.ssh/known_hosts
for addr in unit_worker.find_ip_addresses():
    !ssh-keygen -R {addr}
    check_update_known_hosts(addr)
    !ssh-keyscan -H {addr} >> ~/.ssh/known_hosts

 Ansible でアクセスできることを確認します。

In [ ]:
!ansible {ugroup_name}_{unit_worker.name} -m ping

group_varsに記録されているworkerノードのIPアドレスとノード数を更新します。

In [ ]:
%run scripts/group.py
worker_ipaddresses = unit_worker.find_ip_addresses()
update_group_vars(
    ugroup_name,
    worker_nodes=len(worker_ipaddresses),
    worker_ipaddresses=worker_ipaddresses,
)

### /etc/exportsを更新する

NFSサーバの`/etc/exports`に追加したノードに対応する設定を追加します。

`/etc/exports`の記述を更新します。

In [ ]:
from tempfile import TemporaryDirectory
from pathlib import Path

exports_opts = 'rw,fsid=0,no_root_squash,no_subtree_check,sync,crossmnt'
with TemporaryDirectory() as workdir:
        exports = Path(workdir) / f'{ugroup_name}.exports'
        with exports.open(mode='w') as f:
            if 'nfs_ipaddress' in gvars:
                addr = gvars["manager_ipaddress"]
                f.write(f'/exported/{ugroup_name} {addr}({exports_opts})\n')
            for addr in unit_worker.find_ip_addresses():
                f.write(f'/exported/{ugroup_name} {addr}({exports_opts})\n')
        !cat {exports}
        !ansible {nfs_group} -b -m copy -a \
                'src={exports} dest=/etc/exports.d/ backup=yes'

設定ファイルを読み込ませてエクスポート設定を更新します。

In [ ]:
!ansible {nfs_group} -b -a 'exportfs -r -v'

NFSサーバ側に作成したファイルをNFSクライアント側で参照できることを確認します。

In [ ]:
!ansible {nfs_group} -m file -a 'path=/exported/{ugroup_name}/jupyter/xxx state=touch'
!ansible {nfs_group} -m file -a 'path=/exported/{ugroup_name}/exchange/xxx state=touch'
!ansible {ugroup_name}_{unit_worker.name} -a 'test -f /jupyter/xxx'
!ansible {ugroup_name}_{unit_worker.name} -a 'test -f /exchange/xxx'
!ansible {nfs_group} -m file -a 'path=/exported/{ugroup_name}/jupyter/xxx state=absent'
!ansible {nfs_group} -m file -a 'path=/exported/{ugroup_name}/exchange/xxx state=absent'

### Docker Swarmを更新する

追加したノードをDocker Swarmに追加します。

更新前のworkerノードの一覧を確認します。

In [ ]:
!ansible {ugroup_name}_manager -a 'docker node ls -f role=worker'

workerノードを追加するためのトークンをmanagerノードで取得します。

In [ ]:
out = !ansible {ugroup_name}_manager -a 'docker swarm join-token -q worker' 2> /dev/null
if out[0].find("CHANGED") >= 0 or out[0].find("SUCCESS") >= 0:
    swarm_token = out[1]
    print(swarm_token)
else:
    raise

追加したVCノードをworkerノードとしてDocker Swarmに追加します。

In [ ]:
out = !ansible {ugroup_name}_manager -a \
    'docker node ls -f role=worker --format "{{% raw %}}{{{{.ID}}}}{{% endraw %}}"' 2> /dev/null
exist_nodes = []
for node_id in out[1:]:
    out2 = !ansible {ugroup_name}_manager -a \
        'docker node inspect --format "{{% raw %}}{{{{.Status.Addr}}}}{{% endraw %}}" {node_id}' 2> /dev/null
    exist_nodes.append(out2[1])
manager_ip = gvars['manager_ipaddress']
for addr in unit_worker.find_ip_addresses():
    if addr not in exist_nodes:
        !ansible {addr} -a \
            'docker swarm join --token {swarm_token} {manager_ip}:2377'

更新後のworkerノードの一覧を確認します。

In [ ]:
!ansible {ugroup_name}_manager -a 'docker node ls -f role=worker'

## 管理者ユーザの設定

構築環境のJupyterHubでは講師権限を持つ管理ユーザが操作を行うためのインベントリファイルが配置されています。このインベントリを更新します。

### ユーザの登録

追加したworkerノードに管理者ユーザを登録します。

In [ ]:
%run scripts/cwh.py

for email in gvars.get("teacher_email_list", []):
    name = get_username_from_mail_address(email)
    out = !ansible {ugroup_name}_manager -a 'id -u {name}' 2> /dev/null
    teacher_uid = out[-1]
    out = !ansible {ugroup_name}_manager -a 'id -g {name}' 2> /dev/null
    teacher_gid = out[-1]

    !ansible-playbook -l {ugroup_name}_worker -Dv \
        -e teacher_id={name} -e teacher_uid={teacher_uid} -e teacher_gid={teacher_gid} \
        playbooks/setup-worker-admin-user.yml

### インベントリの更新

管理者に配布している　インベントリに追加したノードを登録します。

In [ ]:
for email in gvars.get("teacher_email_list", []):
    name = get_username_from_mail_address(email)
    !ansible-playbook -Dv -l {ugroup.name}_manager -e teacher_id={name} playbooks/deploy-inventory.yml